In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from data.pointCld_dataset import PointCld_dataset
from prediction import *
from training import * 
from PolyhedronGenerator.polyhedronGenerator import Point
from Network import postprocessing

import matplotlib.pyplot as plt
import os

import plotly.offline as py
py.init_notebook_mode(connected=True)


In [ ]:
path = '1e-4_ResNet_Lr1e-4_epoch13_batchsize500_NVPD21_acceptance0.5_WR8_Adam_train500_validate50_nosphere_Preprocessed_ReLUAvgPool'
params = open_experiment(path)
cfg_path = params['cfg_path']

In [ ]:
predict_dataset = PointCld_dataset(dataset_name='clean_predict_50',
                                    size=50, mode=Mode.PREDICT, cfg_path=cfg_path, augmentation=None, batch_size=1)

predict_loader = torch.utils.data.DataLoader(dataset=predict_dataset,
                                               batch_size=1,
                                               shuffle=False, num_workers=4)
predictor = Prediction(cfg_path, cuda_device_id=6)


In [ ]:
predictor.setup_model(model=Network, convModel='resnet', model_file_name=params['trained_model_name'])

predictor.predict(predict_loader)

'read params new, because predictions folder will be added'
params = open_experiment(path)
cfg_path = params['cfg_path']

In [ ]:
'open a csv file to store execution times'
with open(os.path.join(params["predictions_path"], 'Prediction_Statistics.csv'), mode='w') as file:
                        writer = csv.writer(file, delimiter=',', quotechar='"',
                                            quoting=csv.QUOTE_MINIMAL)
                        writer.writerow(['Name', 'NrPatches', 'ExecutionTime'])

In [ ]:
'run convext optimization and rounding over all above predicted point clouds'

identifiers = [name[:-(len('_relation_matrix_network.npy'))] for name in \
                           os.listdir(params['predictions_path']) if name.endswith('_relation_matrix_network.npy')]

import time
start_time = time.time()

for identifier in identifiers:
    post_processor = postprocessing.Postprocessor(cfg_path, mode='prediction', oracle_m=14)
    post_processor.load_data(name=identifier)
    relation_matrix_cvx = post_processor.apply_MatchLift()
    clusters = post_processor.enforce_consistent_clusters(relation_matrix_cvx)
    post_processor.recover_point_cloud(clusters=clusters)
    post_processor.visualize(jupyter=False, auto_open=False)
    
    'store information to disc'
    with open(os.path.join(params["predictions_path"], 'Prediction_Statistics.csv'), mode='a') as file:
                    writer = csv.writer(file, delimiter=',', quotechar='"',
                                        quoting=csv.QUOTE_MINIMAL)
                    hours, rem = divmod(time.time() - start_time, 3600)
                    minutes, seconds = divmod(rem, 60)
                    writer.writerow([identifier, 
                                     str(relation_matrix_cvx.shape[0]),
                                     str("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))])
